# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 11 2022 12:58PM,248036,21,411606576,"NBTY Global, Inc.",Released
1,Oct 11 2022 12:58PM,248366,19,ACG-2102491533,ACG North America LLC,Released
2,Oct 11 2022 12:56PM,248365,21,50666472,"NBTY Global, Inc.",Released
3,Oct 11 2022 12:46PM,248363,19,ADV80006388,"AdvaGen Pharma, Ltd",Released
4,Oct 11 2022 12:46PM,248363,19,ADV80006389,"AdvaGen Pharma, Ltd",Released
5,Oct 11 2022 12:46PM,248363,19,ADV80006390,"AdvaGen Pharma, Ltd",Released
6,Oct 11 2022 12:46PM,248363,19,ADV80006391,"AdvaGen Pharma, Ltd",Released
7,Oct 11 2022 12:46PM,248363,19,ADV80006392,"AdvaGen Pharma, Ltd",Released
8,Oct 11 2022 12:46PM,248363,19,ADV80006393,"AdvaGen Pharma, Ltd",Released
9,Oct 11 2022 12:46PM,248363,19,ADV80006394,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,248361,Executing,1
26,248363,Released,31
27,248364,Executing,1
28,248365,Released,1
29,248366,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248361,NaN,1.0,NaN
248363,NaN,NaN,31.0
248364,NaN,1.0,NaN
248365,NaN,NaN,1.0
248366,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248034,0.0,0.0,1.0
248036,0.0,0.0,1.0
248264,0.0,10.0,20.0
248268,0.0,43.0,3.0
248277,0.0,12.0,23.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248034,0,0,1
248036,0,0,1
248264,0,10,20
248268,0,43,3
248277,0,12,23


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248034,0,0,1
1,248036,0,0,1
2,248264,0,10,20
3,248268,0,43,3
4,248277,0,12,23


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248034,,,1
1,248036,,,1
2,248264,,10,20
3,248268,,43,3
4,248277,,12,23


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 11 2022 12:58PM,248036,21,"NBTY Global, Inc."
1,Oct 11 2022 12:58PM,248366,19,ACG North America LLC
2,Oct 11 2022 12:56PM,248365,21,"NBTY Global, Inc."
3,Oct 11 2022 12:46PM,248363,19,"AdvaGen Pharma, Ltd"
34,Oct 11 2022 12:46PM,248364,19,"GUSA Granules USA, Inc."
35,Oct 11 2022 12:14PM,248359,15,Lonza
36,Oct 11 2022 12:10PM,248361,15,Lonza
37,Oct 11 2022 11:57AM,248360,10,"Virtue Rx, LLC"
38,Oct 11 2022 11:38AM,248355,16,American International Chemical
39,Oct 11 2022 11:27AM,248352,10,"Virtue Rx, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 11 2022 12:58PM,248036,21,"NBTY Global, Inc.",,,1
1,Oct 11 2022 12:58PM,248366,19,ACG North America LLC,,,1
2,Oct 11 2022 12:56PM,248365,21,"NBTY Global, Inc.",,,1
3,Oct 11 2022 12:46PM,248363,19,"AdvaGen Pharma, Ltd",,,31
4,Oct 11 2022 12:46PM,248364,19,"GUSA Granules USA, Inc.",,1,
5,Oct 11 2022 12:14PM,248359,15,Lonza,,,1
6,Oct 11 2022 12:10PM,248361,15,Lonza,,1,
7,Oct 11 2022 11:57AM,248360,10,"Virtue Rx, LLC",,,1
8,Oct 11 2022 11:38AM,248355,16,American International Chemical,,,1
9,Oct 11 2022 11:27AM,248352,10,"Virtue Rx, LLC",,,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 12:58PM,248036,21,"NBTY Global, Inc.",1,,
1,Oct 11 2022 12:58PM,248366,19,ACG North America LLC,1,,
2,Oct 11 2022 12:56PM,248365,21,"NBTY Global, Inc.",1,,
3,Oct 11 2022 12:46PM,248363,19,"AdvaGen Pharma, Ltd",31,,
4,Oct 11 2022 12:46PM,248364,19,"GUSA Granules USA, Inc.",,1,
5,Oct 11 2022 12:14PM,248359,15,Lonza,1,,
6,Oct 11 2022 12:10PM,248361,15,Lonza,,1,
7,Oct 11 2022 11:57AM,248360,10,"Virtue Rx, LLC",1,,
8,Oct 11 2022 11:38AM,248355,16,American International Chemical,1,,
9,Oct 11 2022 11:27AM,248352,10,"Virtue Rx, LLC",4,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 12:58PM,248036,21,"NBTY Global, Inc.",1,,
1,Oct 11 2022 12:58PM,248366,19,ACG North America LLC,1,,
2,Oct 11 2022 12:56PM,248365,21,"NBTY Global, Inc.",1,,
3,Oct 11 2022 12:46PM,248363,19,"AdvaGen Pharma, Ltd",31,,
4,Oct 11 2022 12:46PM,248364,19,"GUSA Granules USA, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 12:58PM,248036,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Oct 11 2022 12:58PM,248366,19,ACG North America LLC,1.0,NaN,NaN
2,Oct 11 2022 12:56PM,248365,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Oct 11 2022 12:46PM,248363,19,"AdvaGen Pharma, Ltd",31.0,NaN,NaN
4,Oct 11 2022 12:46PM,248364,19,"GUSA Granules USA, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 12:58PM,248036,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Oct 11 2022 12:58PM,248366,19,ACG North America LLC,1.0,0.0,0.0
2,Oct 11 2022 12:56PM,248365,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Oct 11 2022 12:46PM,248363,19,"AdvaGen Pharma, Ltd",31.0,0.0,0.0
4,Oct 11 2022 12:46PM,248364,19,"GUSA Granules USA, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,745033,6.0,1.0,10.0
15,1489808,50.0,68.0,0.0
16,993362,4.0,0.0,0.0
18,248317,1.0,0.0,0.0
19,993443,33.0,1.0,0.0
20,248332,19.0,8.0,5.0
21,744435,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,745033,6.0,1.0,10.0
1,15,1489808,50.0,68.0,0.0
2,16,993362,4.0,0.0,0.0
3,18,248317,1.0,0.0,0.0
4,19,993443,33.0,1.0,0.0
5,20,248332,19.0,8.0,5.0
6,21,744435,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,6.0,1.0,10.0
1,15,50.0,68.0,0.0
2,16,4.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,33.0,1.0,0.0
5,20,19.0,8.0,5.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,6.0
1,15,Released,50.0
2,16,Released,4.0
3,18,Released,1.0
4,19,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,5.0,0.0
Executing,1.0,68.0,0.0,0.0,1.0,8.0,0.0
Released,6.0,50.0,4.0,1.0,33.0,19.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,5.0,0.0
1,Executing,1.0,68.0,0.0,0.0,1.0,8.0,0.0
2,Released,6.0,50.0,4.0,1.0,33.0,19.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,5.0,0.0
1,Executing,1.0,68.0,0.0,0.0,1.0,8.0,0.0
2,Released,6.0,50.0,4.0,1.0,33.0,19.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()